<a href="https://colab.research.google.com/github/seraffin/FailOmen/blob/master/Mariusz/workingNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

# Helper libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from google.colab import drive

# Utilities

In [0]:
# Evaluation function

def evaluation(additionalPredictions, predictions, refYsupervisor, verbose = False):
  
  lenght = len(refYsupervisor)

  failPositions = [[] for y in range(lenght)]

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);

  predictionsTemp = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]


  for i, a in enumerate(predictionsTemp):

    if len(failPositions[i]) != 0:
      for j in range(1 + additionalPredictions):
        argmin = np.argmin(a)
        predictionPositions[i].append(argmin)
        predictionsTemp[i][argmin] = 1

  predictionHits = [[] for y in range(lenght)]

  for i, a in enumerate(failPositions):
    count = 0
    for j, b in enumerate(a):


      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(failPositions) != 0:
      predictionHits[i] = count


  failsCount = 0
  hitsCount = 0
  for i, a in enumerate(refYsupervisor):
    j = 0

    for k, b in enumerate(a):
      if b < 1.0 : j += 1

  #  if j > 0 :
  #    print (i, '.', j, predictionHits[i])


    failsCount += j
    hitsCount += predictionHits[i]

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%')
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')

In [0]:
#Distribution of argmins through all the predictions
  
def argminsDistribution():
  
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j = j + 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
def printFailDistributionStats(y):
  
  failCount = 0
  passCount = 0
  failBuildCount = 0

  for a in y:
    if a[np.argmin(a)] == 0 : failBuildCount += 1
    for b in a:
      if b == 0 : failCount += 1
      if b == 1 : passCount += 1
  print ('failBuildCount', failBuildCount)
  print ('passBuildCount', len(y) - failBuildCount)
  print ('failBuild share ', failBuildCount / len(y) * 100, '%', sep='')
  print ('failCount', failCount)
  print ('passCount', passCount)
  print ('fail share ', failCount / (failCount + passCount) * 100, '%', sep='')

In [0]:
#Distributions of argmins through all the predictions

def countArgminsDistribution(prediction):
  
  i = 1
  tab = [0] * 542
  for a in predictions:
    j = 0
    for b in a:
      if b < 1.0 : j = j + 1

    #print (i, '. ', j, np.argmin(a))
    tab[np.argmin(a)] += 1
    i = i + 1

  i = 0
  distributed_array = []
  for a in tab:
    if a > 0 : 
      print ('position', i, '\targmin count', a)
      distributed_array.append((i,a))
    i += 1
  create_plot(distributed_array)  
  return distributed_array

In [0]:
# Create a list of single member classes positions

def listSingleMemberClassesPositions(refY, printFlag):

  yList = refY.tolist()
  yDistinct = [ele for ind, ele in enumerate(yList) if ele not in yList[:ind]]

  classesPopulation = []
  singleMemberClassesPositions = []
  noSingleMemberClasses = 0

  for i, a in enumerate(yDistinct):
    population = 0
    memberPosition = 0
    for j, b in enumerate(yList):
      if a == b: 
        population += 1
        memberPosition = j

    classesPopulation.append(population)
  #  print(i, population)
    if population == 1: 
      noSingleMemberClasses += 1
      singleMemberClassesPositions.append(memberPosition)

  if printFlag == True: 
    print(noSingleMemberClasses)
  else:
    return singleMemberClassesPositions

In [0]:
def searchForFailed(yList, withpassed_data = False):
  changedRowList = []
  not_add_passed_data = 0
  for row in range (len(yList)):
    if 0 in yList[row]:
      changedRowList.append(row)
      continue
    elif withpassed_data == True:
      if not not_add_passed_data%1:
        changedRowList.append(row)
    not_add_passed_data += 1
      
  return changedRowList

def returnFailedData(xList, yList, changedRowList):
  xFailed = []
  yFailed = []
  
  for row in changedRowList:
    xFailed.append(xList[row])
    yFailed.append(yList[row])
  xFailed = np.array(xFailed)
  yFailed = np.array(yFailed)
  return xFailed, yFailed

In [0]:
# Double the single member classes for stratified sampling

def appendToDataset(refSingleMemberClassesPositions, refDataset):
  
  newDataset = pd.DataFrame(columns=list(refDataset))
  for i, a in enumerate(refSingleMemberClassesPositions):
    newDataset = newDataset.append(refDataset.iloc[a, :])
  
  return refDataset.append(newDataset, ignore_index=True)

In [0]:
def checkGPUpresence():

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

In [0]:
checkGPUpresence()

# Prepare data

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Importing dataset

#dataset = pd.read_csv(dataToML_newConcept.csv', index_col=False)
dataset = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataToML_full2.csv', index_col=False)

In [0]:
# Split data set into parameters and labels

x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

In [0]:
printFailDistributionStats(y)

failBuildCount 712
passBuildCount 4106
failBuild share 14.77791614777916%
failCount 1146
passCount 2610210
fail share 0.04388524582630633%


In [0]:
# Random sampling
'''
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2)

listSingleMemberClassesPositions(yTrain, True)
listSingleMemberClassesPositions(yTest, True)
'''

# Stratified sampling
'''
dataset = appendToDataset(listSingleMemberClassesPositions(y, False), dataset)

x = dataset.iloc[:, 0:23739].values
y = dataset.iloc[:, 23739:24281].values

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=None)

for train_index, test_index in sss.split(x, y):
  xTrain, xTest = x[train_index], x[test_index]
  yTrain, yTest = y[train_index], y[test_index]
  
listSingleMemberClassesPositions(yTrain, True)
listSingleMemberClassesPositions(yTest, True)
'''

# Reduce overpopulated class with fully passed builds

xReduced, yReduced = returnFailedData(x, y, searchForFailed(y,withpassed_data=False))
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(xReduced, yReduced, test_size = 0.2)

# Prepare and train model

In [0]:
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(15175, activation="sigmoid", input_dim=23739, kernel_initializer="uniform"))
# Adding the second hidden layer
model.add(Dense(15175, activation = "sigmoid", kernel_initializer="uniform"))
# Adding the output layer
model.add(Dense(542, activation="sigmoid", kernel_initializer="uniform"))

In [0]:
model.compile(optimizer = tf.train.MomentumOptimizer(learning_rate = 0.01, momentum = 0.4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(xTrain, yTrain, batch_size = 20, epochs = 15)

Epoch 1/15
569/569 [==============================] - 17s 30ms/step - loss: 0.4939 - acc: 0.8015
Epoch 2/15
569/569 [==============================] - 6s 10ms/step - loss: 0.1306 - acc: 0.9963
Epoch 3/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0774 - acc: 0.9971
Epoch 4/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0578 - acc: 0.9971
Epoch 5/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0474 - acc: 0.9971
Epoch 6/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0411 - acc: 0.9971
Epoch 7/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0368 - acc: 0.9971
Epoch 8/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0337 - acc: 0.9971
Epoch 9/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0313 - acc: 0.9971
Epoch 10/15
569/569 [==============================] - 6s 10ms/step - loss: 0.0295 - acc: 0.9971
Epoch 11/15
569/569 [=================

# Evaluate

In [0]:
predictions = model.predict(xTest)

In [0]:
#Distributions of argmins through all the predictions
  
def argminsDistribution(): 
    i = 1
    tab = [0] * 542
    for a in predictions:
      j = 0
      for b in a:
        if b < 1.0 : j = j + 1

      #print (i, '. ', j, np.argmin(a))
      tab[np.argmin(a)] += 1
      i = i + 1

    i = 0
    for a in tab:
      if a > 0 : print ('position', i, '\targmin count', a)
      i += 1

In [0]:
#failsCount()

In [0]:
argminsDistribution()

position 462 	argmin count 143


In [0]:
# Evaluate by probability - if the percentage parameter is set to 0.5 than 50% 
# of predictions will be taken into cosideration. When the strongest prediction 
# in given build is 0.9 then predictions from 0.9 to 0.95 will be 
# taken into consideration, if 0.8 than from 0.8 to 0.9

def evaluation2(percentage, predictions, refYsupervisor, verbose=False):
  
  lenght = len(refYsupervisor)
  failPositions = [[] for y in range(lenght)]
  failPredictions = predictions.copy()
  predictionPositions = [[] for y in range(lenght)]
  predictionHits = [0] * lenght
  validPredictionsCount = [0] * lenght

  for i, a in enumerate(refYsupervisor):

    for j, b in enumerate(a):
      if b == 0 : failPositions[i].append(j);
  
  for i, a in enumerate(predictions):
    probabilityLimit = np.amin(a) + ((1 - np.amin(a)) * percentage)

    for j, b in enumerate(a):
      if b <= probabilityLimit: 
        predictionPositions[i].append(j)
        validPredictionsCount[i] += 1
        
  for i, a in enumerate(failPositions):
    count = 0
    
    for b in a:

      for c in predictionPositions[i]:
        if c == b : count += 1

    if len(a) != 0:
      predictionHits[i] = count

  hitsCount = 0
  failsCount = 0
  
  for i in range(lenght):
   # print(predictionHits[i])
    hitsCount += predictionHits[i]
    failsCount += len(failPositions[i])  

  print('Percentage of fails predicted', hitsCount / failsCount * 100, '%',
        '//Average of valid predictions', sum(validPredictionsCount)/len(validPredictionsCount))
  
  if verbose == True:
    for i in range(lenght):
      print(i ,len(failPositions[i]), len(predictionPositions[i]), predictionHits[i], sep='\t')

In [0]:
for i in range(14):
  print(i + 1, ' ', end='')
  evaluation(i, predictions, yTest)

1  Percentage of fails predicted 2.307692307692308 %
2  Percentage of fails predicted 4.615384615384616 %
3  Percentage of fails predicted 8.846153846153847 %
4  Percentage of fails predicted 11.153846153846155 %
5  Percentage of fails predicted 11.538461538461538 %
6  Percentage of fails predicted 13.076923076923078 %
7  Percentage of fails predicted 13.846153846153847 %
8  Percentage of fails predicted 16.153846153846153 %
9  Percentage of fails predicted 17.692307692307693 %
10  Percentage of fails predicted 19.230769230769234 %
11  Percentage of fails predicted 20.0 %
12  Percentage of fails predicted 21.153846153846153 %
13  Percentage of fails predicted 21.153846153846153 %
14  Percentage of fails predicted 22.692307692307693 %


In [0]:
for i in range(21):
  print(round(0.05 * i, 3), ' ', end='')
  evaluation2(0.05 * i, predictions, yTest)

0.0  Percentage of fails predicted 2.307692307692308 % //Average of valid predictions 1.0
0.05  Percentage of fails predicted 2.307692307692308 % //Average of valid predictions 1.0
0.1  Percentage of fails predicted 2.307692307692308 % //Average of valid predictions 1.0
0.15  Percentage of fails predicted 2.307692307692308 % //Average of valid predictions 1.0
0.2  Percentage of fails predicted 2.6923076923076925 % //Average of valid predictions 1.132867132867133
0.25  Percentage of fails predicted 8.076923076923077 % //Average of valid predictions 2.7762237762237763
0.3  Percentage of fails predicted 8.846153846153847 % //Average of valid predictions 3.0
0.35  Percentage of fails predicted 8.846153846153847 % //Average of valid predictions 3.0
0.4  Percentage of fails predicted 8.846153846153847 % //Average of valid predictions 3.3846153846153846
0.45  Percentage of fails predicted 13.076923076923078 % //Average of valid predictions 6.251748251748252
0.5  Percentage of fails predicted 